# Character-wise RNNs

In [1]:
# prepare packages
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [2]:
# open text
anna_text_fn = '../data/anna.txt'
def open_text_file(fn):
    with open(anna_text_fn, 'r') as f:
        t = f.read()
    return t
text = open_text_file(anna_text_fn)
text[:50]

'Chapter 1\n\n\nHappy families are all alike; every un'

## Preprocessings

In [3]:
# preprocessing
# get unique characters
vocab = set(text)
print('vocabulary size: ', len(vocab))

vocabulary size:  83


In [4]:
# mark ids to characters
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = {i: c for c, i in vocab_to_int.items()}

In [5]:
# encode anna text in np.array
encoded = np.array( [vocab_to_int[c] for c in text], dtype=np.int32 )
encoded[:50]

array([33, 51, 68, 46, 77, 41, 13, 45, 73, 50, 50, 50, 69, 68, 46, 46, 61,
       45,  6, 68, 71, 40,  8, 40, 41, 42, 45, 68, 13, 41, 45, 68,  8,  8,
       45, 68,  8, 40, 25, 41, 62, 45, 41, 11, 41, 13, 61, 45, 23, 15])

### Making batches

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: the number of sequences per batch(sequence length)
       n_steps: Number of sequence steps per batch(how many sequence needed for single batch?)
    '''
    #print('Input arr length: ', len(arr))
    
    # get number of chracters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # keep only enough chracters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    #print('cliped arr length: ', len(arr))
    #print('characters_per_batch: ', characters_per_batch)
    #print('batch size: ', n_batches)
    
    # reshape
    arr = arr.reshape( (n_seqs, -1) )
    #print('reshaped arr: ', arr.shape)
    
    for n in range(0, arr.shape[1], n_steps):
        # the features
        x = arr[:, n:n + n_steps]
        # targets, x shifted by 1
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded, 10, 50)

In [8]:
x, y = next(batches)
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[33 51 68 46 77 41 13 45 73 50]
 [45 68 71 45 15 24 77 45 79 24]
 [11 40 15 70 50 50 39 37 41 42]
 [15 45 30 23 13 40 15 79 45 51]
 [45 40 77 45 40 42 36 45 42 40]
 [45  2 77 45 16 68 42 50 24 15]
 [51 41 15 45 28 24 71 41 45  6]
 [62 45 55 23 77 45 15 24 16 45]
 [77 45 40 42 15 63 77 70 45 29]
 [45 42 68 40 30 45 77 24 45 51]]

y
 [[51 68 46 77 41 13 45 73 50 50]
 [68 71 45 15 24 77 45 79 24 40]
 [40 15 70 50 50 39 37 41 42 36]
 [45 30 23 13 40 15 79 45 51 40]
 [40 77 45 40 42 36 45 42 40 13]
 [ 2 77 45 16 68 42 50 24 15  8]
 [41 15 45 28 24 71 41 45  6 24]
 [45 55 23 77 45 15 24 16 45 42]
 [45 40 42 15 63 77 70 45 29 51]
 [42 68 40 30 45 77 24 45 51 41]]


## Building the model

### Inputs

In [9]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
    '''
    inputs = tf.placeholder( tf.int32, [batch_size, num_steps], name='inputs' )
    targets = tf.placeholder( tf.int32, [batch_size, num_steps], name='targets' )
    
    # keep probability placeholder for drop out layers
    keep_prob = tf.placeholder( tf.float32, name='keep_prob' )
    
    return inputs, targets, keep_prob

### LSTM Cell

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size
    '''
    # build lstm cell with dropout
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    # stack up multiple LSTM layers
    cell = tf.contrib.rnn.MultiRNNCell( [build_cell(lstm_size, keep_prob) for _ in range(num_layers)] )
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN output

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    '''
    # reshape output so it's a bunch of rows, one row for each step for each sequence.
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable( tf.truncated_normal([in_size, out_size], stddev=0.1) )
        softmax_b = tf.Variable(  tf.zeros(out_size))
    
    # logits
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits=logits, name='predictions')
    
    return out, logits

### Training loss

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets        
    '''
    # one-hot encoded targets and reshape to match logitsm one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # softmax cross entrophy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

In [13]:
def build_optimizer(loss, learnin_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer    
    '''
    # optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate=learnin_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

In [14]:
class CharRNN:    
    def __init__(self, num_classes, batch_size=64, num_steps=50,
                 lstm_size=128, num_layers=2, learning_rate=0.001, 
                 grad_clip=5, sampling=False):
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

### Hyperparameters
Here I'm defining the hyperparameters for the network.
* batch_size - Number of sequences running through the network in one pass.
* num_steps - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn * more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* lstm_size - The number of units in the hidden layers.
* num_layers - Number of hidden LSTM layers to use
* learning_rate - Learning rate for training
* keep_prob - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

In [15]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Training

In [16]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4209...  0.3951 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3325...  0.1530 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.8526...  0.1530 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 5.2568...  0.1490 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.3578...  0.1510 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.9851...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.8226...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.6760...  0.1640 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.5467...  0.1510 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.4759...  0.1490 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.4065...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.3933...  0.1480 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 3.0997...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.0879...  0.1490 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.0928...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.0929...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0609...  0.1510 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0717...  0.1510 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0749...  0.1490 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.0536...  0.1560 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 3.0615...  0.1500 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.0634...  0.1490 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 3.0507...  0.1520 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 3.0491...  0.1510 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.5040...  0.1520 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.5013...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.5007...  0.1690 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.5042...  0.1510 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.5241...  0.1510 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.4802...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.4751...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.4849...  0.1490 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.4822...  0.1520 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.5042...  0.1510 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.4770...  0.1501 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.4754...  0.1570 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 303...  Training loss: 2.2680...  0.1590 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.2755...  0.1530 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.2770...  0.1520 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.2965...  0.1540 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.2876...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.2520...  0.1570 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.2772...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.2881...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 311...  Training loss: 2.2550...  0.1540 sec/batch
Epoch: 2/20...  Training Step: 312...  Training loss: 2.2474...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 313...  Training loss: 2.2503...  0.1500 sec/batch
Epoch: 2/20...  Training Step: 314...  Training loss: 2.2175...  0.1510 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 403...  Training loss: 2.1251...  0.1544 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.1314...  0.1555 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.1561...  0.1572 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.1309...  0.1542 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.1149...  0.1555 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.1030...  0.1485 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 2.1248...  0.1520 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 2.1595...  0.1500 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 2.1119...  0.1530 sec/batch
Epoch: 3/20...  Training Step: 412...  Training loss: 2.0987...  0.1500 sec/batch
Epoch: 3/20...  Training Step: 413...  Training loss: 2.1140...  0.1520 sec/batch
Epoch: 3/20...  Training Step: 414...  Training loss: 2.1510...  0.1540 sec/batch
Epoch: 3/20...  

...

Epoch: 19/20...  Training Step: 3567...  Training loss: 1.2406...  0.1510 sec/batch
Epoch: 19/20...  Training Step: 3568...  Training loss: 1.2649...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3569...  Training loss: 1.2227...  0.1530 sec/batch
Epoch: 19/20...  Training Step: 3570...  Training loss: 1.2045...  0.1490 sec/batch
Epoch: 19/20...  Training Step: 3571...  Training loss: 1.2360...  0.1520 sec/batch
Epoch: 19/20...  Training Step: 3572...  Training loss: 1.2455...  0.1510 sec/batch
Epoch: 19/20...  Training Step: 3573...  Training loss: 1.2480...  0.1530 sec/batch
Epoch: 19/20...  Training Step: 3574...  Training loss: 1.2373...  0.1490 sec/batch
Epoch: 19/20...  Training Step: 3575...  Training loss: 1.2336...  0.1560 sec/batch
Epoch: 19/20...  Training Step: 3576...  Training loss: 1.2368...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3577...  Training loss: 1.2480...  0.1620 sec/batch
Epoch: 19/20...  Training Step: 3578...  Training loss: 1.2623...  0.1500 se

Epoch: 19/20...  Training Step: 3665...  Training loss: 1.2247...  0.1510 sec/batch
Epoch: 19/20...  Training Step: 3666...  Training loss: 1.2229...  0.1510 sec/batch
Epoch: 19/20...  Training Step: 3667...  Training loss: 1.2428...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3668...  Training loss: 1.2224...  0.1510 sec/batch
Epoch: 19/20...  Training Step: 3669...  Training loss: 1.2402...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3670...  Training loss: 1.2300...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3671...  Training loss: 1.2475...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3672...  Training loss: 1.2531...  0.1500 sec/batch
Epoch: 19/20...  Training Step: 3673...  Training loss: 1.2269...  0.1550 sec/batch
Epoch: 19/20...  Training Step: 3674...  Training loss: 1.2543...  0.1510 sec/batch
Epoch: 19/20...  Training Step: 3675...  Training loss: 1.2137...  0.1490 sec/batch
Epoch: 19/20...  Training Step: 3676...  Training loss: 1.2424...  0.1630 se

Epoch: 20/20...  Training Step: 3763...  Training loss: 1.3617...  0.1700 sec/batch
Epoch: 20/20...  Training Step: 3764...  Training loss: 1.2456...  0.1510 sec/batch
Epoch: 20/20...  Training Step: 3765...  Training loss: 1.2233...  0.1540 sec/batch
Epoch: 20/20...  Training Step: 3766...  Training loss: 1.2570...  0.1500 sec/batch
Epoch: 20/20...  Training Step: 3767...  Training loss: 1.2169...  0.1510 sec/batch
Epoch: 20/20...  Training Step: 3768...  Training loss: 1.1962...  0.1480 sec/batch
Epoch: 20/20...  Training Step: 3769...  Training loss: 1.2298...  0.1490 sec/batch
Epoch: 20/20...  Training Step: 3770...  Training loss: 1.2270...  0.1500 sec/batch
Epoch: 20/20...  Training Step: 3771...  Training loss: 1.2334...  0.1510 sec/batch
Epoch: 20/20...  Training Step: 3772...  Training loss: 1.2307...  0.1490 sec/batch
Epoch: 20/20...  Training Step: 3773...  Training loss: 1.2205...  0.1530 sec/batch
Epoch: 20/20...  Training Step: 3774...  Training loss: 1.2331...  0.1500 se

Epoch: 20/20...  Training Step: 3861...  Training loss: 1.2089...  0.1530 sec/batch
Epoch: 20/20...  Training Step: 3862...  Training loss: 1.2081...  0.1550 sec/batch
Epoch: 20/20...  Training Step: 3863...  Training loss: 1.2213...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3864...  Training loss: 1.2195...  0.1520 sec/batch
Epoch: 20/20...  Training Step: 3865...  Training loss: 1.2271...  0.1550 sec/batch
Epoch: 20/20...  Training Step: 3866...  Training loss: 1.2130...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3867...  Training loss: 1.2164...  0.1550 sec/batch
Epoch: 20/20...  Training Step: 3868...  Training loss: 1.2202...  0.1510 sec/batch
Epoch: 20/20...  Training Step: 3869...  Training loss: 1.2346...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3870...  Training loss: 1.2293...  0.1510 sec/batch
Epoch: 20/20...  Training Step: 3871...  Training loss: 1.2131...  0.1530 sec/batch
Epoch: 20/20...  Training Step: 3872...  Training loss: 1.2390...  0.1580 se

Epoch: 20/20...  Training Step: 3959...  Training loss: 1.2148...  0.1500 sec/batch
Epoch: 20/20...  Training Step: 3960...  Training loss: 1.2188...  0.1560 sec/batch


### Saved checkpoints

In [17]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200_l512.ckpt"
all_mo

## Sampling
Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.

In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [20]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i3960_l512.ckpt'

In [21]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i3960_l512.ckpt
Farther that in the
days of her subjects of the first teash, and she had seen their soul,
because he was in his wife, and he had been and she seemed to see her that the
man had that lade to be simply angry. At the same as a maturing of husband
to bring the best tervors. He had the prayer was so ashes of simply than
he would be delighted; and that he would no more her soft and all, and
went to his brother, and was several to the proform of his wheels and
attention of anyence.

"If you can did not certain you and so so sat down," she asked.

"I have been all right, to be at the poor of all the chorse. Thyse confries
is not at abternating in that subject, that I can't both her one arouse,
have the party of the figure of the such thought.
I think it's a sensable condition of that is sometices another
to home, that he has suppressed. That's the sace well all were to be in sicture
in which it he have to be controlled to he